# Nuclio - Generator function

## Environment

In [1]:
# nuclio: ignore
import nuclio

### Configurations

In [2]:
import os
import datetime

In [22]:
# nuclio: ignore
# Set initial timestamp as 7 days ago
os.environ['INITIAL_TS'] = str(int((datetime.datetime.now()-datetime.timedelta(days=1)).timestamp()))
print(os.getenv('INITIAL_TS', 0))

1560956279


In [23]:
%%nuclio config 

# Trigger
spec.triggers.secs.kind = "cron"
spec.triggers.secs.attributes.interval = "10s"

# Base image
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.triggers.secs.kind to 'cron'
%nuclio: setting spec.triggers.secs.attributes.interval to '10s'
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Set configuration file for function pulling

In [24]:
# nuclio: ignore
os.environ['local_configurations_path'] = os.path.join(os.path.dirname(os.getcwd()), 'configurations', 'metrics_configuration.yaml')
os.environ['shared_configurations_path'] = os.path.join('/', 'v3io', 'bigdata', 'netops_configurations')
os.environ['webapi_configuration_path'] = os.path.join('/', 'bigdata', 'netops_configurations', 'metrics_configuration.yaml')
os.environ['function_configuration_dir'] = os.path.join('/', 'configurations')

!mkdir ${shared_configurations_path}
!cp ${local_configurations_path} -t ${shared_configurations_path}

mkdir: cannot create directory ‘/v3io/bigdata/netops_configurations’: File exists


In [25]:
%nuclio env -c METRICS_CONFIGURATION_FILEPATH=/configurations/metrics_configuration.yaml
%nuclio env -l METRICS_CONFIGURATION_FILEPATH=../configurations/metrics_configuration.yaml

%nuclio: setting 'METRICS_CONFIGURATION_FILEPATH' environment variable


In [26]:
%%nuclio cmd -c

# Pull configuration file to function
apt-get update && apt-get install -y wget
mkdir -p ${function_configuration_dir}
wget -O ${METRICS_CONFIGURATION_FILEPATH} --header "x-v3io-session-key: ${V3IO_ACCESS_KEY}" http://${V3IO_WEBAPI_SERVICE_HOST}:8081${webapi_configuration_path}

### Setups

In [8]:
%%nuclio cmd

# Utils
pip install pyarrow
pip install pyyaml --upgrade
pip install pandas
pip install pytimeparse

# Igz DB
pip install v3io_frames --upgrade

# Function
pip install -i https://test.pypi.org/simple/ v3io-generator
pip install faker

nuclio-jupyter 0.7.2 has requirement tornado<6,>=5, but you'll have tornado 6.0.2 which is incompatible.
  Found existing installation: PyYAML 5.1
Cannot uninstall 'PyYAML'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
Requirement already up-to-date: v3io_frames in /User/.pythonlibs/lib/python3.6/site-packages (0.5.6)
Looking in indexes: https://test.pypi.org/simple/


### Variables

In [27]:
%%nuclio env

# DB Config
V3IO_FRAMESD=${V3IO_FRAMESD}
V3IO_USERNAME=${V3IO_USERNAME}
V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

# Deployment
SAVE_DEPLOYMENT=1
DEPLOYMENT_TABLE=netops_devices

# Metrics

# Parquet
# SAVE_TO=/v3io/bigdata/netops_metrics_parquet
SAVE_TO=netops_metrics

INITIAL_TIMESTAMP=${INITIAL_TS}
SECS_TO_GENERATE=10

# Save as
SAVE_TO_TSDB=1

%nuclio: setting 'V3IO_FRAMESD' environment variable
%nuclio: setting 'V3IO_USERNAME' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable
%nuclio: setting 'SAVE_DEPLOYMENT' environment variable
%nuclio: setting 'DEPLOYMENT_TABLE' environment variable
%nuclio: setting '# SAVE_TO' environment variable
%nuclio: setting 'SAVE_TO' environment variable
%nuclio: setting 'INITIAL_TIMESTAMP' environment variable
%nuclio: setting 'SECS_TO_GENERATE' environment variable
%nuclio: setting 'SAVE_TO_TSDB' environment variable


%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line
%nuclio: cannot find "=" in line


## Function

In [28]:
# import os # Already imported earlier
import time
import yaml
import pandas as pd
import itertools

# DB Connection
import v3io_frames as v3f

# Data generator
from v3io_generator import metrics_generator, deployment_generator

### Helper functions

In [29]:
def _create_deployment():
    print('creating deployment')
    # Create meta-data factory
    dep_gen = deployment_generator.deployment_generator()
    faker=dep_gen.get_faker()

    # Design meta-data
    dep_gen.add_level(name='company',number=2,level_type=faker.company)
    dep_gen.add_level('data_center',number=2,level_type=faker.street_name)
    dep_gen.add_level('device',number=2,level_type=faker.msisdn)

    # Create meta-data
    deployment_df = dep_gen.generate_deployment()
    return deployment_df

In [30]:
def _is_deployment_exist(path):
    # Checking shared path for the devices table
    return os.path.exists(f'/v3io/bigdata/{path}')

In [31]:
def _get_deployment_from_kv(client, path):
    print(f'Retrieving deployment from {path}')
    context.logger.debug(f'Retrieving deployment from {path}')
    # Read the devices table from our KV store
    deployment_df = client.read(backend='kv', table=path)
    
    # Reset index to column
    deployment_df.index.name = 'device'
    deployment_df = deployment_df.reset_index()
    return deployment_df

In [32]:
def _save_deployment_to_kv(path, df, client=v3f.Client('framesd:8081')):
    # Save deployment to our KV store
    client.write(backend='kv', table='netops_devices',dfs=df, index_cols=['device'])

In [33]:
def get_or_create_deployment(path, save_to_cloud=False, client=None):
    if client and _is_deployment_exist(path):
        # Get deployment from KV
        deployment_df = _get_deployment_from_kv(client, path)
    else:
        # Create deployment
        deployment_df = _create_deployment()
        
        if client and save_to_cloud:
            _save_deployment_to_kv(path, deployment_df, client)

    return deployment_df

In [34]:
def set_indexes(df):
    df = df.set_index(['timestamp', 'company', 'data_center', 'device'])
    return df

In [35]:
def save_metrics_to_tsdb(context, metrics: pd.DataFrame):
    print('Saving metrics to TSDB')
    
    context.v3f.write('tsdb', context.metrics_table, metrics)

In [36]:
def save_metrics_to_parquet(context, metrics):
    print('Saving metrics to Parquet')
    df = pd.concat(itertools.chain(metrics))
    
    # Need to fix timestamps from ns to ms if we write to parquet
    df = df.reset_index()
    df['timestamp'] = df.loc[:, 'timestamp'].astype('datetime64[ms]')
    
    # Fix indexes
    df = set_indexes(df)
    
    # Save parquet
    first_timestamp = df.index[0][0].strftime('%Y%m%dT%H%M%S')
    last_timestamp = df.index[-1][0].strftime('%Y%m%dT%H%M%S')
    filename = first_timestamp + '-' + last_timestamp + '.parquet'
    print(filename)
    filepath = os.path.join(context.metrics_table, filename)
    print(filepath)
    with open(filepath, 'wb+') as f:
        df.to_parquet(f)

In [37]:
def is_deployment_initialized(context):
    return hasattr(context, 'metric_generator')

### Init context

In [38]:
def init_context(context):
    
    # Get saving configuration
    save_to_tsdb = (int(os.getenv('SAVE_TO_TSDB', 1)) == 1)
    
    # Set metrics table
    metrics_table = os.getenv('SAVE_TO', 'netops_metrics')
    setattr(context, 'metrics_table', metrics_table) 

    # TSDB Based demo
    if save_to_tsdb:
        context.logger.debug('Saving to TSDB')
        # Create our DB client
        client = v3f.Client(address='framesd:8081', container='bigdata')
        
        # Create TSDB table if needed
        client.create('tsdb', metrics_table, attrs={'rate': '1/s'}, if_exists=1)
        
        # Set saving function
        setattr(context, 'write', save_metrics_to_tsdb)
    
    # Parquet based demo
    else:
        context.logger.debug('Saving to Parquet')
        # Set empty client for verification purposes
        client = None
          
        # Create saving directory
        filepath = os.path.join(metrics_table)
        if not os.path.exists(filepath):
            os.makedirs(filepath)
        
        # Set saving function
        setattr(context, 'write', save_metrics_to_parquet)
    
          
    # Set batch endtime
    secs_to_generate = os.getenv('SECS_TO_GENERATE', 10)
    setattr(context, 'secs_to_generate', secs_to_generate)
    
     
    
    # Generate or create deployment
    deployment_df = get_or_create_deployment(os.environ['DEPLOYMENT_TABLE'], os.environ['SAVE_DEPLOYMENT'], client)
    
    deployment_df['cpu_utilization'] = 70
    deployment_df['latency'] = 0
    deployment_df['packet_loss'] = 0
    deployment_df['throughput'] = 290
    deployment_df.head()
    
    # Get metrics configuration
    with open(os.getenv('METRICS_CONFIGURATION_FILEPATH', '/configurations/metrics_configuration.yaml'), 'r') as f:
        metrics_configuration = yaml.load(f)
        
    # Create metrics generator
    initial_timestamp = int(os.getenv('INITIAL_TIMESTAMP', time.time()))
    met_gen = metrics_generator.Generator_df(metrics_configuration, 
                                             user_hierarchy=deployment_df, 
                                             initial_timestamp=initial_timestamp)
    setattr(context, 'metric_generator', met_gen)
    
    # Set client
    setattr(context, 'v3f', client)


### Handler

In [39]:
def handler(context, event):
       
    # Create metrics generator based on YAML configuration and deployment
    metrics = context.metric_generator.generate_range(start_time=datetime.datetime.now(),
                                     end_time=datetime.datetime.now()+datetime.timedelta(seconds=int(context.secs_to_generate)),
                                     as_df=True,
                                     as_iterator=True)
    
    # Save Generated metrics
    context.write(context, metrics)

## Test

In [40]:
# nuclio: ignore
init_context(context)
event = nuclio.Event(body='')
output = handler(context, event)
output

Retrieving deployment from netops_devices


/conda/lib/python3.6/site-packages/ipykernel_launcher.py:57: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


Saving metrics to TSDB


# Deploy

In [85]:
%nuclio deploy -p netops -n generator -c

[nuclio.deploy] 2019-06-20 06:05:47,211 (info) Building processor image
[nuclio.deploy] 2019-06-20 06:05:53,269 (info) Pushing image
[nuclio.deploy] 2019-06-20 06:05:53,270 (info) Build complete
[nuclio.deploy] 2019-06-20 06:05:57,365 (info) Function deploy complete
[nuclio.deploy] 2019-06-20 06:05:57,370 done updating generator, function address: 18.185.111.133:31908
%nuclio: function deployed
